# 1. Parameters

In [1]:
# Defaults

simulation_dir = 'simulations/unset'
reference_file = 'simulations/reference/reference.fa.gz'

In [2]:
from pathlib import Path

simulation_dir_path = Path(simulation_dir)

case_name = str(simulation_dir_path.name)

simulation_data_path = simulation_dir_path / 'simulated_data'
simulated_variants_file = simulation_data_path / 'haplotypes.vcf.gz'

case_name = str(simulation_dir_path.name)
index_reads_path = simulation_dir_path / 'index-reads'
index_assemblies_path = simulation_dir_path / 'index-assemblies'

# 2. Load simulated variants (VCF)

In [3]:
import vcf
import pandas as pd
import time

reader = vcf.Reader(filename=str(simulated_variants_file))
simulated_df = pd.DataFrame([vars(r) for r in reader])
simulated_df = simulated_df[['CHROM', 'POS', 'REF', 'samples']]
simulated_df.head(5)

,CHROM,POS,REF,samples
0,chrom0,12,G,"[Call(sample=reference, CallData(GT=0, GQ=4414..."
1,chrom0,13,T,"[Call(sample=reference, CallData(GT=0, GQ=4414..."
2,chrom0,38,C,"[Call(sample=reference, CallData(GT=0, GQ=4414..."
3,chrom0,42,G,"[Call(sample=reference, CallData(GT=0, GQ=4414..."
4,chrom0,54,T,"[Call(sample=reference, CallData(GT=0, GQ=4414..."


## 2.1. Construct sample:variant identifiers

In [4]:
before = time.time()

# Explode dataframe so that we have one row per sample
simulated_df_exploded = simulated_df.explode('samples')
simulated_df_exploded['SAMPLE'] = simulated_df_exploded['samples'].apply(lambda x: x.sample)

# Extrat ALT for each sample
simulated_df_exploded['ALT'] = simulated_df_exploded['samples'].apply(lambda x: x.gt_bases)

# Only keep mutations/those where REF and ALT are different
simulated_df_exploded = simulated_df_exploded[simulated_df_exploded['REF'] != simulated_df_exploded['ALT']]

# Create SPDI-like identifier with Sample name for comparison (SAMPLE:CHROM:POS:REF:ALT)
simulated_df_exploded['ID'] = simulated_df_exploded.apply(
    lambda x: f"{x['SAMPLE']}:{x['CHROM']}:{x['POS']}:{x['REF']}:{x['ALT']}", axis='columns')

simulated_variants = simulated_df_exploded['ID'].reset_index(drop=True)
expected_sample_variants = set(simulated_variants)

after = time.time()

print(f'There are {len(expected_sample_variants)} expected sample/variant pairs like: '
      f'{list(simulated_variants[0:3])}')
print(f'Took {(after - before)/60:0.1f} minutes')

There are 10109 expected sample/variant pairs like: ['SH08-001:chrom0:12:G:A', 'SH09-29:chrom0:13:T:A', 'SH10-30:chrom0:13:T:A']
Took 0.0 minutes


In [5]:
simulated_df_exploded

,CHROM,POS,REF,samples,SAMPLE,ALT,ID
0,chrom0,12,G,"Call(sample=SH08-001, CallData(GT=1, GQ=441453))",SH08-001,A,SH08-001:chrom0:12:G:A
1,chrom0,13,T,"Call(sample=SH09-29, CallData(GT=1, GQ=441453))",SH09-29,A,SH09-29:chrom0:13:T:A
1,chrom0,13,T,"Call(sample=SH10-30, CallData(GT=1, GQ=441453))",SH10-30,A,SH10-30:chrom0:13:T:A
1,chrom0,13,T,"Call(sample=SH14-001, CallData(GT=1, GQ=441453))",SH14-001,A,SH14-001:chrom0:13:T:A
1,chrom0,13,T,"Call(sample=SH14-010, CallData(GT=1, GQ=441453))",SH14-010,A,SH14-010:chrom0:13:T:A
...,...,...,...,...,...,...,...
794,chrom0,10803,G,"Call(sample=SH12-002, CallData(GT=1, GQ=441453))",SH12-002,A,SH12-002:chrom0:10803:G:A
794,chrom0,10803,G,"Call(sample=SH12-007, CallData(GT=1, GQ=441453))",SH12-007,A,SH12-007:chrom0:10803:G:A
795,chrom0,10816,A,"Call(sample=SH10-002, CallData(GT=1, GQ=441453))",SH10-002,T,SH10-002:chrom0:10816:A:T
796,chrom0,10821,A,"Call(sample=SH12-011, CallData(GT=1, GQ=441453))",SH12-011,G,SH12-011:chrom0:10821:A:G


## 1.2. Load reference genome for use with positive/negative calculations

In [6]:
import gzip
from Bio import SeqIO

with gzip.open(reference_file, mode='rt') as f:
    sequences = list(SeqIO.parse(f, 'fasta'))
    reference_length = len(sequences[0])
    
sample_names = set(simulated_df_exploded.groupby('SAMPLE').agg({'SAMPLE': 'count'}).index)
sample_names = sample_names - {'reference'}
number_samples = len(sample_names)

print(f'Reference length: {reference_length}')
print(f'Number samples: {number_samples}')

Reference length: 10834
Number samples: 59


# 3. Load detected variants

## 3.1. Load from reads index

In [7]:
from typing import Set
import genomics_data_index.api as gdi

def get_sample_variant_idenifiers(index_dir: Path) -> Set[str]:
    db = gdi.GenomicsDataIndex.connect(index_dir)
    q = db.samples_query()

    actual_sample_variants = set()
    for sample in q.tolist():
        sample_features = q.isa(sample).features_summary().reset_index()
        sample_features_set = set(sample_features['Mutation'].apply(lambda x: f"{sample}:{x}"))
        actual_sample_variants.update(sample_features_set)
    return actual_sample_variants

before = time.time()
actual_sample_variants_reads = get_sample_variant_idenifiers(index_reads_path)
after = time.time()

print(f'There are {len(actual_sample_variants_reads)} actual sample/variant pairs (reads)')
print(f'Took {(after - before)/60:0.1f} minutes')
print(f'Actual variants look like: {list(actual_sample_variants_reads)[0:5]}')

There are 9183 actual sample/variant pairs (reads)
Took 0.0 minutes
Actual variants look like: ['SH14-027:chrom0:2790:GA:TG', 'SH13-003:chrom0:6935:C:G', 'SH10-015:chrom0:916:T:C', 'SH11-002:chrom0:9567:T:C', 'SH14-010:chrom0:3939:A:G']


## 2.2. Load from assemblies index

In [8]:
before = time.time()
actual_sample_variants_assemblies = get_sample_variant_idenifiers(index_assemblies_path)
after = time.time()

print(f'There are {len(actual_sample_variants_assemblies)} actual sample/variant pairs (assemblies)')
print(f'Took {(after - before)/60:0.1f} minutes')

Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7f91fca41ac0> but it is already set


There are 10026 actual sample/variant pairs (assemblies)
Took 0.0 minutes


# 3. Compare expected/actual variants

## 3.1. Compare with reads index

In [9]:
def compare_expected_actual(name: str, actual_sample_variants: Set[str]) -> pd.DataFrame:
    number_expected = len(expected_sample_variants)
    number_actual = len(actual_sample_variants)

    true_positives = actual_sample_variants & expected_sample_variants
    false_negatives = expected_sample_variants - actual_sample_variants
    false_positives = actual_sample_variants - expected_sample_variants
    # I cannot get true negatives since I would need to know the total number of negatives (i.e., all possible 
    #  variants with respect to the reference genome that were not simulated). This would be a finite, but 
    #  very very large number (and I haven't worked out how to calculate it).
    # For example, one negative is Sample:1:A:T, another negative is Sample:1:AG:TT, and so on for the entire 
    # length of the genome.
    # true_negatives = set()

    sensitivity = len(true_positives) / (len(true_positives) + len(false_negatives))
    precision = len(true_positives) / (len(true_positives) + len(false_positives))
    f1_score = 2 * len(true_positives) / (2 * len(true_positives) + len(false_positives) + len(false_negatives))

    # Since true_negatives are a very large number, then for all intents and purposes
    # specificity will be very very close to 1. So instead of trying to calculate it
    # I just call it ~1, but it's also not very useful because of this.
    #specificity = len(true_negatives) / (len(true_negatives) + len(false_positives))
    specificity = '~1 (not calculated)'

    print(f'Sensitivity: {sensitivity:0.4f}')
    print(f'Specificity: {specificity}')
    print(f'Precision: {precision:0.4f}')
    print(f'F1 Score: {f1_score:0.4f}')
    
    comparison_df = pd.DataFrame([{
        'Name': name,
        'True Positives': len(true_positives),
        'True Negatives': pd.NA,
        'False Positives': len(false_positives),
        'False Negatives': len(false_negatives),
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'Precision': precision,
        'F1 Score': f1_score,
    }])
    
    data = {
        'tp': true_positives,
        'fp': false_positives,
        'fn': false_negatives
    }
    
    return comparison_df, data
    
comparison_reads_df, data_reads = compare_expected_actual(name=f'{case_name} reads',
                                                          actual_sample_variants=actual_sample_variants_reads)
comparison_reads_df

Sensitivity: 0.8490
Specificity: ~1 (not calculated)
Precision: 0.9347
F1 Score: 0.8898


,Name,True Positives,True Negatives,False Positives,False Negatives,Sensitivity,Specificity,Precision,F1 Score
0,unset reads,8583,<NA>,600,1526,0.849045,~1 (not calculated),0.934662,0.889799


## 3.2. Compare with assemblies index

In [10]:
comparison_assemblies_df, data_assemblies = compare_expected_actual(name=f'{case_name} assemblies',
                                                                    actual_sample_variants=actual_sample_variants_assemblies)
comparison_assemblies_df

Sensitivity: 0.9918
Specificity: ~1 (not calculated)
Precision: 1.0000
F1 Score: 0.9959


,Name,True Positives,True Negatives,False Positives,False Negatives,Sensitivity,Specificity,Precision,F1 Score
0,unset assemblies,10026,<NA>,0,83,0.991789,~1 (not calculated),1.0,0.995878


## 3.3. Combine results

In [11]:
results_df = pd.concat([comparison_reads_df, comparison_assemblies_df])
results_df

,Name,True Positives,True Negatives,False Positives,False Negatives,Sensitivity,Specificity,Precision,F1 Score
0,unset reads,8583,<NA>,600,1526,0.849045,~1 (not calculated),0.934662,0.889799
0,unset assemblies,10026,<NA>,0,83,0.991789,~1 (not calculated),1.000000,0.995878


In [12]:
results_df_output = simulation_dir_path / 'variants-comparison.tsv'
results_df.to_csv(results_df_output, sep='\t', index=False)